# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [6]:
def simulate_single_flow(g, s, t, func_traj, difusao):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            try:
                # Escolhe aleatoriamente um dos sucessores.
                m = func_traj(g, n) ##
            except IndexError:
                continue
            
            # Deixa de ser dono do insumo.
            g.nodes[n]['owner'] = difusao ###
            
            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

            
        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [7]:
def simulate_successful_flow(g, s, t, func_traj, difusao):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, func_traj, difusao)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [8]:
def simulate_all_flows(g,func_traj, difusao):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, func_traj, difusao)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não precisa entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Média de *closeness simulado* e *betweenness simulado* para muitas repetições da simulação acima.

In [9]:
# TIMES = 1000
# func_traj = random_geodesic_successor
# difusao = False

def simul(TIMES, func_traj, difusao):
    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, func_traj, difusao)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES
        
    df = pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
    'closeness analítico': [cc[n] for n in g.nodes],
    'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
    'betweenness analítico': [bc[n] for n in g.nodes],
    })
    return df
#     return df.sort_values(coluna, ascending=0)#.describe()

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [10]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

Construção de data frame só para comparar mais facilmente.

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [11]:
from scipy import stats

In [12]:
bm = simul(100, random_geodesic_successor, False)
bmc = bm["closeness simulado"]
bmb = bm["betweenness simulado"]
bm.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.115751,0.411834,0.411834
std,0.136863,0.136222,0.074293,0.074293
min,0.000000,0.000000,0.285714,0.285714
25%,0.010989,0.011538,0.359211,0.359211
50%,0.087912,0.089011,0.400000,0.400000
75%,0.131868,0.132115,0.474713,0.474713
max,0.521978,0.521813,0.560000,0.560000


In [13]:
bm.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.521813,0.560000,0.560000,medici
3,0.260073,0.256868,0.466667,0.466667,guadagni
2,0.212454,0.208077,0.482759,0.482759,albizzi
5,0.142857,0.142857,0.388889,0.388889,salviati
8,0.120879,0.121374,0.400000,0.400000,bischeri
13,0.115385,0.115659,0.437500,0.437500,barbadori
7,0.091575,0.092308,0.482759,0.482759,tornabuon
9,0.086081,0.089011,0.482759,0.482759,ridolfi
14,0.087912,0.087857,0.388889,0.388889,castellan
11,0.075092,0.077363,0.424242,0.424242,strozzi


In [14]:
bm.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.521813,0.560000,0.560000,medici
2,0.212454,0.208077,0.482759,0.482759,albizzi
7,0.091575,0.092308,0.482759,0.482759,tornabuon
9,0.086081,0.089011,0.482759,0.482759,ridolfi
3,0.260073,0.256868,0.466667,0.466667,guadagni
13,0.115385,0.115659,0.437500,0.437500,barbadori
11,0.075092,0.077363,0.424242,0.424242,strozzi
8,0.120879,0.121374,0.400000,0.400000,bischeri
5,0.142857,0.142857,0.388889,0.388889,salviati
14,0.087912,0.087857,0.388889,0.388889,castellan


In [15]:
gd = simul(100, random_geodesic_successor, True)
gdc = gd["closeness simulado"]
gdb = gd["betweenness simulado"]
gd.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.311604,0.411834,0.411834
std,0.136863,0.364331,0.074293,0.074293
min,0.000000,0.000000,0.285714,0.285714
25%,0.010989,0.027280,0.359211,0.359211
50%,0.087912,0.248297,0.400000,0.400000
75%,0.131868,0.361484,0.474713,0.474713
max,0.521978,1.389066,0.560000,0.560000


In [16]:
gd.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,1.389066,0.560000,0.560000,medici
3,0.260073,0.689121,0.466667,0.466667,guadagni
2,0.212454,0.568516,0.482759,0.482759,albizzi
5,0.142857,0.406593,0.388889,0.388889,salviati
13,0.115385,0.316374,0.437500,0.437500,barbadori
8,0.120879,0.314451,0.400000,0.400000,bischeri
9,0.086081,0.249560,0.482759,0.482759,ridolfi
7,0.091575,0.248297,0.482759,0.482759,tornabuon
14,0.087912,0.230165,0.388889,0.388889,castellan
11,0.075092,0.207363,0.424242,0.424242,strozzi


In [17]:
gd.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,1.389066,0.560000,0.560000,medici
2,0.212454,0.568516,0.482759,0.482759,albizzi
7,0.091575,0.248297,0.482759,0.482759,tornabuon
9,0.086081,0.249560,0.482759,0.482759,ridolfi
3,0.260073,0.689121,0.466667,0.466667,guadagni
13,0.115385,0.316374,0.437500,0.437500,barbadori
11,0.075092,0.207363,0.424242,0.424242,strozzi
8,0.120879,0.314451,0.400000,0.400000,bischeri
5,0.142857,0.406593,0.388889,0.388889,salviati
14,0.087912,0.230165,0.388889,0.388889,castellan


In [18]:
stats.ttest_ind(bmc,gdc)

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [19]:
stats.ttest_ind(bmb,gdb)

Ttest_indResult(statistic=-1.9501460890469202, pvalue=0.061235317369256012)

In [20]:
pt = simul(100, random_path_successor, False)
ptc = pt["closeness simulado"]
ptb = pt["betweenness simulado"]
pt.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.247930,0.411834,0.243321
std,0.136863,0.187862,0.074293,0.026640
min,0.000000,0.000000,0.285714,0.201148
25%,0.010989,0.071429,0.359211,0.231081
50%,0.087912,0.251374,0.400000,0.242804
75%,0.131868,0.369808,0.474713,0.254946
max,0.521978,0.626209,0.560000,0.309266


In [21]:
pt.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.626209,0.560000,0.309266,medici
3,0.260073,0.409121,0.466667,0.267260,guadagni
14,0.087912,0.381978,0.388889,0.257531,castellan
8,0.120879,0.372802,0.400000,0.248335,bischeri
11,0.075092,0.366813,0.424242,0.237060,strozzi
9,0.086081,0.362582,0.482759,0.251374,ridolfi
7,0.091575,0.331154,0.482759,0.252717,tornabuon
12,0.021978,0.251374,0.350000,0.225101,peruzzi
13,0.115385,0.243407,0.437500,0.239895,barbadori
2,0.212454,0.230659,0.482759,0.257174,albizzi


In [22]:
pt.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.626209,0.560000,0.309266,medici
3,0.260073,0.409121,0.466667,0.267260,guadagni
14,0.087912,0.381978,0.388889,0.257531,castellan
2,0.212454,0.230659,0.482759,0.257174,albizzi
7,0.091575,0.331154,0.482759,0.252717,tornabuon
9,0.086081,0.362582,0.482759,0.251374,ridolfi
8,0.120879,0.372802,0.400000,0.248335,bischeri
5,0.142857,0.142857,0.388889,0.242804,salviati
10,0.000000,0.000000,0.368421,0.240240,acciaiuol
13,0.115385,0.243407,0.437500,0.239895,barbadori


In [23]:
stats.ttest_ind(bmc,ptc)

Ttest_indResult(statistic=8.2692594638841808, pvalue=5.3409211074804037e-09)

In [24]:
stats.ttest_ind(bmb,ptb)

Ttest_indResult(statistic=-2.2060854745783298, pvalue=0.035751529748774873)

In [25]:
pd1 = simul(100, random_path_successor, True)
pdc = pd1["closeness simulado"]
pdb = pd1["betweenness simulado"]
pd1.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.566436,0.411834,0.251731
std,0.136863,0.159570,0.074293,0.021527
min,0.000000,0.260549,0.285714,0.204766
25%,0.010989,0.419505,0.359211,0.234142
50%,0.087912,0.613736,0.400000,0.256035
75%,0.131868,0.683791,0.474713,0.266014
max,0.521978,0.822363,0.560000,0.281829


In [26]:
pd1.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.822363,0.560000,0.266598,medici
3,0.260073,0.740549,0.466667,0.266009,guadagni
9,0.086081,0.704396,0.482759,0.278500,ridolfi
7,0.091575,0.684231,0.482759,0.281829,tornabuon
11,0.075092,0.683352,0.424242,0.266018,strozzi
8,0.120879,0.651923,0.400000,0.259827,bischeri
14,0.087912,0.615934,0.388889,0.253491,castellan
2,0.212454,0.613736,0.482759,0.256035,albizzi
13,0.115385,0.578462,0.437500,0.265835,barbadori
12,0.021978,0.522143,0.350000,0.252638,peruzzi


In [27]:
pd1.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
7,0.091575,0.684231,0.482759,0.281829,tornabuon
9,0.086081,0.704396,0.482759,0.278500,ridolfi
6,0.521978,0.822363,0.560000,0.266598,medici
11,0.075092,0.683352,0.424242,0.266018,strozzi
3,0.260073,0.740549,0.466667,0.266009,guadagni
13,0.115385,0.578462,0.437500,0.265835,barbadori
8,0.120879,0.651923,0.400000,0.259827,bischeri
2,0.212454,0.613736,0.482759,0.256035,albizzi
14,0.087912,0.615934,0.388889,0.253491,castellan
12,0.021978,0.522143,0.350000,0.252638,peruzzi


In [28]:
stats.ttest_ind(bmc,pdc)

Ttest_indResult(statistic=8.0166081272970882, pvalue=9.9205371993118863e-09)

In [29]:
stats.ttest_ind(bmb,pdb)

Ttest_indResult(statistic=-8.3195215628016239, pvalue=4.726605654039869e-09)

In [30]:
tt = simul(100, random_trail_successor, False)
ttc = tt["closeness simulado"]
ttb = tt["betweenness simulado"]
tt.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.248110,0.411834,0.238989
std,0.136863,0.196089,0.074293,0.024745
min,0.000000,0.000000,0.285714,0.195803
25%,0.010989,0.071429,0.359211,0.224747
50%,0.087912,0.267253,0.400000,0.238953
75%,0.131868,0.338489,0.474713,0.253194
max,0.521978,0.707143,0.560000,0.289865


In [31]:
tt.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.707143,0.560000,0.289865,medici
3,0.260073,0.405714,0.466667,0.261381,guadagni
14,0.087912,0.373846,0.388889,0.226692,castellan
8,0.120879,0.340769,0.400000,0.227098,bischeri
9,0.086081,0.336209,0.482759,0.238953,ridolfi
11,0.075092,0.328571,0.424242,0.234440,strozzi
7,0.091575,0.323791,0.482759,0.245008,tornabuon
13,0.115385,0.267253,0.437500,0.263181,barbadori
2,0.212454,0.266813,0.482759,0.271533,albizzi
12,0.021978,0.228681,0.350000,0.241453,peruzzi


In [32]:
tt.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,0.707143,0.560000,0.289865,medici
2,0.212454,0.266813,0.482759,0.271533,albizzi
13,0.115385,0.267253,0.437500,0.263181,barbadori
3,0.260073,0.405714,0.466667,0.261381,guadagni
7,0.091575,0.323791,0.482759,0.245008,tornabuon
12,0.021978,0.228681,0.350000,0.241453,peruzzi
5,0.142857,0.142857,0.388889,0.239335,salviati
9,0.086081,0.336209,0.482759,0.238953,ridolfi
11,0.075092,0.328571,0.424242,0.234440,strozzi
8,0.120879,0.340769,0.400000,0.227098,bischeri


In [33]:
stats.ttest_ind(bmc,ttc)

Ttest_indResult(statistic=8.5488484790904433, pvalue=2.7180316482295321e-09)

In [34]:
stats.ttest_ind(bmb,ttb)

Ttest_indResult(statistic=-2.1470063945283377, pvalue=0.040597178571309969)

In [35]:
td = simul(100, random_trail_successor, True)
tdc = td["closeness simulado"]
tdb = td["betweenness simulado"]
td.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,0.682220,0.411834,0.241009
std,0.136863,0.280863,0.074293,0.017550
min,0.000000,0.265659,0.285714,0.200893
25%,0.010989,0.412005,0.359211,0.229192
50%,0.087912,0.690714,0.400000,0.249278
75%,0.131868,0.895027,0.474713,0.252718
max,0.521978,1.256978,0.560000,0.261963


In [36]:
td.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,1.256978,0.560000,0.253707,medici
9,0.086081,0.930714,0.482759,0.261963,ridolfi
7,0.091575,0.924835,0.482759,0.260248,tornabuon
3,0.260073,0.910110,0.466667,0.256169,guadagni
11,0.075092,0.879945,0.424242,0.249278,strozzi
14,0.087912,0.828736,0.388889,0.242822,castellan
8,0.120879,0.790220,0.400000,0.251729,bischeri
2,0.212454,0.690714,0.482759,0.250371,albizzi
13,0.115385,0.598571,0.437500,0.251289,barbadori
12,0.021978,0.591429,0.350000,0.235843,peruzzi


In [37]:
td.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
9,0.086081,0.930714,0.482759,0.261963,ridolfi
7,0.091575,0.924835,0.482759,0.260248,tornabuon
3,0.260073,0.910110,0.466667,0.256169,guadagni
6,0.521978,1.256978,0.560000,0.253707,medici
8,0.120879,0.790220,0.400000,0.251729,bischeri
13,0.115385,0.598571,0.437500,0.251289,barbadori
2,0.212454,0.690714,0.482759,0.250371,albizzi
11,0.075092,0.879945,0.424242,0.249278,strozzi
14,0.087912,0.828736,0.388889,0.242822,castellan
12,0.021978,0.591429,0.350000,0.235843,peruzzi


In [38]:
stats.ttest_ind(bmc,tdc)

Ttest_indResult(statistic=8.6667972509101272, pvalue=2.0503900633686561e-09)

In [39]:
stats.ttest_ind(bmb,tdb)

Ttest_indResult(statistic=-7.0283220707793959, pvalue=1.2082931268193442e-07)

In [40]:
wt = simul(100, random_walk_successor, False)
wtc = wt["closeness simulado"]
wtb = wt["betweenness simulado"]
wt.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,2.081341,0.411834,0.036290
std,0.136863,1.173852,0.074293,0.001797
min,0.000000,0.729780,0.285714,0.033831
25%,0.010989,1.177555,0.359211,0.034899
50%,0.087912,2.457253,0.400000,0.035819
75%,0.131868,2.509203,0.474713,0.037482
max,0.521978,5.067473,0.560000,0.040089


In [41]:
wt.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,5.067473,0.560000,0.035580,medici
3,0.260073,3.330330,0.466667,0.035354,guadagni
11,0.075092,2.526429,0.424242,0.035819,strozzi
9,0.086081,2.519670,0.482759,0.036518,ridolfi
7,0.091575,2.498736,0.482759,0.034470,tornabuon
8,0.120879,2.497692,0.400000,0.034866,bischeri
14,0.087912,2.490879,0.388889,0.033831,castellan
2,0.212454,2.457253,0.482759,0.034931,albizzi
12,0.021978,1.631154,0.350000,0.036978,peruzzi
13,0.115385,1.620604,0.437500,0.034351,barbadori


In [42]:
wt.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
4,0.000000,0.729780,0.285714,0.040089,pazzi
10,0.000000,0.769231,0.368421,0.038849,acciaiuol
5,0.142857,1.585879,0.388889,0.037745,salviati
1,0.000000,0.756978,0.325581,0.037740,lambertes
0,0.000000,0.738022,0.333333,0.037225,ginori
12,0.021978,1.631154,0.350000,0.036978,peruzzi
9,0.086081,2.519670,0.482759,0.036518,ridolfi
11,0.075092,2.526429,0.424242,0.035819,strozzi
6,0.521978,5.067473,0.560000,0.035580,medici
3,0.260073,3.330330,0.466667,0.035354,guadagni


In [43]:
stats.ttest_ind(bmc,wtc)

Ttest_indResult(statistic=19.571829732491786, pvalue=7.1505949957070255e-18)

In [44]:
stats.ttest_ind(bmb,wtb)

Ttest_indResult(statistic=-6.4419932527816517, pvalue=5.6187493045090685e-07)

In [45]:
wd = simul(100, random_walk_successor, True)
wdc = wd["closeness simulado"]
wdb = wd["betweenness simulado"]
wd.describe()

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado
count,15.000000,15.000000,15.000000,15.000000
mean,0.115751,2.363103,0.411834,0.155144
std,0.136863,1.652598,0.074293,0.013002
min,0.000000,0.578516,0.285714,0.129000
25%,0.010989,1.133626,0.359211,0.144826
50%,0.087912,2.390549,0.400000,0.155746
75%,0.131868,2.868709,0.474713,0.163470
max,0.521978,6.845495,0.560000,0.173902


In [46]:
wd.sort_values('betweenness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
6,0.521978,6.845495,0.560000,0.173598,medici
3,0.260073,4.449286,0.466667,0.160952,guadagni
14,0.087912,3.079725,0.388889,0.153336,castellan
2,0.212454,3.011374,0.482759,0.156790,albizzi
8,0.120879,2.726044,0.400000,0.155741,bischeri
11,0.075092,2.668077,0.424242,0.155779,strozzi
9,0.086081,2.524176,0.482759,0.172099,ridolfi
7,0.091575,2.390549,0.482759,0.173902,tornabuon
5,0.142857,1.802143,0.388889,0.144008,salviati
12,0.021978,1.642692,0.350000,0.145644,peruzzi


In [47]:
wd.sort_values('closeness simulado', ascending=0)

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
7,0.091575,2.390549,0.482759,0.173902,tornabuon
6,0.521978,6.845495,0.560000,0.173598,medici
9,0.086081,2.524176,0.482759,0.172099,ridolfi
13,0.115385,1.473846,0.437500,0.165988,barbadori
3,0.260073,4.449286,0.466667,0.160952,guadagni
2,0.212454,3.011374,0.482759,0.156790,albizzi
11,0.075092,2.668077,0.424242,0.155779,strozzi
10,0.000000,0.578516,0.368421,0.155746,acciaiuol
8,0.120879,2.726044,0.400000,0.155741,bischeri
14,0.087912,3.079725,0.388889,0.153336,castellan


In [48]:
stats.ttest_ind(bmc,wdc)

Ttest_indResult(statistic=13.181200659540984, pvalue=1.57625730718508e-13)

In [49]:
stats.ttest_ind(bmc,wdb)

Ttest_indResult(statistic=-4.5683253625303522, pvalue=9.0197488154398364e-05)

A hiopotese nula (as duas amostras sao iguais) é rejeitada quando o p-value é menor que 0.05.

Dessa forma, a única situação onde não há rejeição da hipótese nula (tanto no closeness quanto no betweeness) é no caso de geodésica com difusão, uma vez que seu p-valor é maior que 5%.

Com isso, é possível afirmar que há diferença entre as simulações.